In [1]:
import random
import multiprocessing

import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F


### Contents
1. System settings
2. Dataset & Dataloader & Augmentation
3. Model
4. Optimizer & Loss func
5. Training & Inference

In [2]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        """
        Args:
            num_classes (int): 정답 label의 classification 갯수
        """
        
        super().__init__() # nn.module을 상속받음
        # input size : (b x 3 x 227 x 227)
        
        self.net = nn.Sequential( # nn.Sequential : model의 층을 연속적으로 쌓아주는 함수
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),  # (b x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  # section 3.3에 나와 있는 hyperparameter 값 사용  
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (b x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (b x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (b x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (b x 256 x 6 x 6)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False), # inplace=True : 계산한 값을 변수에 덮어씌움 / True 시 오류가 나서 False로 통일
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )
        self.init_bias()  # 아래 함수 참고

    def init_bias(self):
        for layer in self.net: # self.net에 정의된 layer에서
            if isinstance(layer, nn.Conv2d): # 해당 layer가 Conv2D layer 라면 
                nn.init.normal_(layer.weight, mean=0, std=0.01) # Normal distribution으로 weight initialization 진행
                nn.init.constant_(layer.bias, 0) # Bias를 모두 0으로 initialization

        nn.init.constant_(self.net[4].bias, 1) # 2번째 Conv2D layer bias를 1로 초기화
        nn.init.constant_(self.net[10].bias, 1) # 4번째 Conv2D layer의 bias를 1로 초기화
        nn.init.constant_(self.net[12].bias, 1) # 5번째 Conv2D layer의 bias를 1로 초기화

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 256 * 6 * 6)  # view는 tensor shape를 바꿔줌, FC layer로 넘겨주기 위해 output을 1차원으로 펴주는 역할
        return self.classifier(x)

In [3]:
def seed_everything(seed): # Reproducibility를 위한 seed 고정 작업
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

seed_everything(42)

# Settings

use_cuda = torch.cuda.is_available() # if the system supports CUDA -> True
device = torch.device("cuda" if use_cuda else "cpu")

# Dataset & augmentation

data_transform = transforms.Compose([
            transforms.Resize((227, 227)), # image tensor size를 227 x 227로 변환 (input에 넣어주기 위함)
            transforms.ToTensor(), # input을 tensor type으로 전환
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # image tensor 값들을 normalization
])

training_data = datasets.CIFAR10(
    root="data", # Download=True일 시, 데이터를 다운받을 경로 // Download=False일 시, 데이터가 존재하는 경로
    train=True,  # True -> Training set에서 data를 가져옴, False -> Test set에서 data를 가져옴
    download=False, # Data가 시스템 내부에 존재하는 지 여부
    transform=data_transform, # 위에서 정의한 data transformation을 적용
)

test_data = datasets.CIFAR10(
    root="data",
    train=False, 
    download=False,
    transform=data_transform,
)

batch_size = 128 # 논문에 나와 있는 batch size

train_dataloader = DataLoader(dataset=training_data, # dataset from which to load the data 
    batch_size=batch_size,
    pin_memory=use_cuda, # True 시, CUDA memory에 tensor를 올려 놓음
    num_workers=multiprocessing.cpu_count()//2, # 원래는 tuning해야 하지만, 일반적인 cpu 개수로 worker 할당
    shuffle=True) # True 시 epoch마다 data reshuffle 후 sampling 진행
    
test_dataloader = DataLoader(dataset=test_data,
    batch_size=batch_size,
    pin_memory=use_cuda,
    num_workers=multiprocessing.cpu_count()//2,
    shuffle=False) # trained model의 정확한 performance 비교를 위해 섞지 않음 

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break



Shape of X [N, C, H, W]: torch.Size([128, 3, 227, 227])
Shape of y: torch.Size([128]) torch.int64


In [4]:
# -- model

model = AlexNet().to(device) # Tensor를 지정한 device에서 연산 진행하도록 하기 위해, to(device)를 붙임 
model = torch.nn.DataParallel(model) # 여러 GPU 상에서 병렬 연산 진행

print(model)

DataParallel(
  (module): AlexNet(
    (net): Sequential(
      (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
      (1): ReLU()
      (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): ReLU()
      (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU()
      (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU()
      (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): 

In [5]:
optimizer = optim.SGD(
        params=model.parameters(), # optimize할 parameter set 지정
        lr=0.01, # learning rate 지정
        momentum=0.9, # 논문에 나와 있는 SGD momentum 값 
        weight_decay=0.0005) # Overfit을 방지하기 위해, weight의 절대적 규모를 전체적으로 감소시키는 역할

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1) # (step_size) epoch 마다 (gamma) 배씩 learning rate를 감소시켜주는 schedular

In [13]:
# Training

torch.cuda.empty_cache()

for epoch in range(100):
    model.train() # Training과 Inference 시 다르게 작동하는 layer(e.x. Dropout)를 처리해주기 위해, model을 training mode로 전환

    loss = 0

    for idx, train_batch in enumerate(train_dataloader):

        inputs, labels = train_batch
        inputs = inputs.to(device) # model의 input
        labels = labels.to(device) # model의 정답 label
        
        optimizer.zero_grad() # pytorch는 backward 시 gradients 값들을 누적하기 때문에, zero_grad() 를 통해 매 step마다 초기화해주어야 한다.

        outs = model(inputs) # model의 predictions
        loss = F.cross_entropy(outs, labels) # loss value는 cross entropy로 값을 구함

        loss.backward() # 'Require_grad=True'인 모든 tensor에 대한 미분 계산
        optimizer.step() # 계산된 loss를 바탕으로, parameter들을 update

        if idx % 100 == 0: # 100 iteration 마다 진행
            loss, current = loss.item(), idx * len(X) # loss.item() : loss tensor 내 값 반환
            size = len(train_dataloader.dataset)      # current / size : 현재 학습 진행 상황
            print(f" Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
    lr_scheduler.step() # stepLR 진행 
        
    model.eval() # 매 epoch 마다, test set으로 model evaluation을 진행하기 위해서 evaluation mode로 진행

    test_loss, correct = 0, 0

    with torch.no_grad(): # no_grad() 는 자동으로 gradient 추적하는 것을 막음 -> 메모리 사용량을 줄이고 연산속도 높이기 위함
        size = len(test_dataloader.dataset)
        num_batches = len(test_dataloader)

        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device) # X : input, y : 정답 label
            test_pred = model(X)
            test_loss += F.cross_entropy(test_pred, y).item()
            correct += (torch.argmax(test_pred, dim=-1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error in epoch {epoch+1} : \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


 Train loss: 2.318789  [    0/50000]
 Train loss: 2.310215  [ 1600/50000]
 Train loss: 2.343995  [ 3200/50000]
 Train loss: 2.369611  [ 4800/50000]
Test Error in epoch 0 : 
 Accuracy: 10.0%, Avg loss: 2.333501 

 Train loss: 2.269061  [    0/50000]
 Train loss: 2.329641  [ 1600/50000]
 Train loss: 2.375973  [ 3200/50000]
 Train loss: 2.301568  [ 4800/50000]
Test Error in epoch 1 : 
 Accuracy: 10.0%, Avg loss: 2.333501 

 Train loss: 2.377271  [    0/50000]
 Train loss: 2.340114  [ 1600/50000]
 Train loss: 2.387733  [ 3200/50000]
 Train loss: 2.326668  [ 4800/50000]
Test Error in epoch 2 : 
 Accuracy: 10.0%, Avg loss: 2.333501 

 Train loss: 2.381385  [    0/50000]
 Train loss: 2.309510  [ 1600/50000]
 Train loss: 2.330097  [ 3200/50000]
 Train loss: 2.367663  [ 4800/50000]
Test Error in epoch 3 : 
 Accuracy: 10.0%, Avg loss: 2.333501 

 Train loss: 2.326844  [    0/50000]
 Train loss: 2.354359  [ 1600/50000]
 Train loss: 2.377092  [ 3200/50000]
 Train loss: 2.358562  [ 4800/50000]
Test

## 소감
 먼저, pytorch를 이용해서 모델을 raw하게 구현하는 것은 이번이 처음이어서 코드를 구성하는 데 좀 헤맸던 것 같다. 기존에 내가 pytorch를 공부했을 때 배웠던 것들을 적용해볼 생각이었는데, 아직 실력이 모자라 내 맘대로 구현하기가 쉽지 않았다. 그래서 기본적인 틀을 구현하되 pytorch 기본적인 부분에 대한 설명을 잘 이해하는 것을 목표로 삼았다.

 아쉬웠던 점
 1. GPU programming
    논문에 나와 있는 대로, 특정 층에서 GPU interaction을 통한 연산을 구현해보려고 여러 방법을 찾아봤으나, 쉽지 않았고, 시간을 더 투자해봤자 좋은 결과를 내지 못할 것 같아 포기했다. 따라서 기본 모델 클래스 구현과 Engineering 관점에서 기초적으로 어떤 것들을 적용할 수 있는 지 생각하는 데 초점을 잡았다. 이 부분은 나중에 조금 더 공부해보고 전해보고 싶다.
 2. LR schedular 구현
    논문에서는 epoch 당 validation set에 대한 accuracy가 plateaus에 다다랐다고 생각되었을 때 learning rate를 0.1씩 감소한다고 나와있다. 그런데 plateaus에 대한 기준을 잡는 게 쉽지 않아, 간단하게 stepLR로 schedular를 짰다. 